In [7]:
from multicamera_acquisition.paths import DATA_DIR
from multicamera_acquisition.acquisition import AcquisitionLoop
from multicamera_acquisition.interfaces.camera_uvc import UVCCamera as Camera
from multicamera_acquisition.writer import FFMPEG_Writer as Writer
from multicamera_acquisition.video_utils import count_frames
import time
from datetime import datetime
import multiprocessing as mp

In [8]:
save_location = DATA_DIR / 'tests'
fps = 210
duration = 10000 # msec

In [9]:
camera_config = Camera.default_camera_config().copy()
camera_config["name"] = "test_camera"
camera_config["fps"] = fps
file_name = f'{camera_config["name"]}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

In [10]:
write_queue = mp.Queue()
writer_config = Writer.default_writer_config(fps).copy()
writer_config["camera_name"] = camera_config["name"]
writer = Writer(write_queue, 
                video_file_name=save_location / f"{file_name}.mp4",
                metadata_file_name=save_location / f"{file_name}.csv",
                config=writer_config)

In [11]:
acq_config = AcquisitionLoop.default_acq_loop_config().copy()
acq_config["max_frames_to_acquire"] = int(fps * duration / 1000)
acq_loop = AcquisitionLoop(write_queue=write_queue,
                           display_queue=None,
                           camera_device_index=None,
                           camera_config=camera_config,
                           acq_loop_config=acq_config,
                           fps=fps)

In [12]:
# start recording
writer.start()

acq_loop.start()
acq_loop.await_process.wait()
acq_loop.await_main_thread.set()
acq_loop.await_process.wait()

time.sleep(duration/1000)
acq_loop.stop()

acq_loop.join(timeout=60)
writer.join(timeout=60)

/Users/brianxu/Documents/github/multicamera_acquisition/multicamera_acquisition/interfaces/camera_azure.py:18: UserWarning: pyk4a not installed.  Azure cameras will not be available.
  warnings.warn("pyk4a not installed.  Azure cameras will not be available.")
Traceback (most recent call last):
  File "/Users/brianxu/Documents/github/multicamera_acquisition/multicamera_acquisition/interfaces/camera_uvc.py", line 192, in _create_uvc_cam
    self.cam = self.system.Capture(devices[self.device_index]['uid'])
  File "uvc_bindings.pyx", line 539, in uvc_bindings.Capture.__init__
  File "uvc_bindings.pyx", line 595, in uvc_bindings.Capture._init_device
uvc_bindings.OpenError: Could not open device. Error: Access denied

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brianxu/Documents/github/multicamera_acquisition/multicamera_acquisition/interfaces/camera_uvc.py", line 169, in init
    self._create_uvc_cam()
  File "/User

KeyboardInterrupt: 

In [ ]:
assert writer.video_file_name.exists()
time.sleep(1)
count_frames(str(writer.video_file_name))

1895

In [ ]:
import cv2

cap = cv2.VideoCapture(str(writer.video_file_name), cv2.CAP_FFMPEG)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('frame', frame)    